In [2]:
import os
import math

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import cPickle

In [4]:
root = 'cifar-10-batches-py/'

In [5]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = cPickle.load(fo)
    return data

In [6]:
files = [
'batches.meta',
'data_batch_1',
'data_batch_2',
'data_batch_3',
'data_batch_4',
'data_batch_5',
'test_batch']

In [7]:
b = {}

for each in files:  
    path = root + each
    b[each] = unpickle(path)

In [8]:
b['batches.meta'].keys()

['num_cases_per_batch', 'label_names', 'num_vis']

In [9]:
b['data_batch_1'].keys()

['data', 'labels', 'batch_label', 'filenames']

### Train data prep

In [10]:
x_train = []
y_train = []

for i in files[1:-1]:
    [x_train.append(j) for j in b[i]['data']]
    [y_train.append(j) for j in b[i]['labels']]
    
train_mean = np.mean(x_train, 1)
train_std = np.std(x_train, 1)

x_train = np.array(
    [(x_train[i]-train_mean[i])/train_std[i] for i in range(50000)])
y_train = np.array(y_train)

In [11]:
x_train.shape

(50000, 3072)

In [12]:
y_train[1]

9

In [13]:
b['batches.meta']['label_names']

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [14]:
x_train_rs = np.array([i.reshape(3, 32, 32).transpose([1, 2, 0]) for i in x_train])

In [15]:
x_train_rs.shape

(50000, 32, 32, 3)

In [16]:
y_train.shape

(50000,)

### Test data prep

In [17]:
x_test = b['test_batch']['data']
y_test = b['test_batch']['labels']

test_mean = np.mean(x_test, 1)
test_std = np.std(x_test, 1)

x_test = np.array(
    [(x_test[i]-test_mean[i])/test_std[i] for i in range(10000)])
y_test = np.array(y_test)

In [18]:
x_test.shape

(10000, 3072)

In [19]:
x_test_rs = np.array([i.reshape(3, 32, 32).transpose([1, 2, 0]) for i in x_test])

In [20]:
x_test_rs.shape

(10000, 32, 32, 3)

In [21]:
y_test.shape

(10000,)

In [22]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


### Non Binary network

In [23]:
tf.set_random_seed(2018)

In [24]:
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_true = tf.placeholder(tf.float32, [None, 10])

In [25]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [26]:
def create_biases(num_filters):
    return tf.Variable(tf.ones([num_filters]))

In [27]:
def create_convolutional_layer(input,
                               num_input_channels,
                               conv_filter_size,
                               num_filters):  
    
    weights = create_weights([conv_filter_size, conv_filter_size,
                              num_input_channels, num_filters])
   
    biases = create_biases(num_filters)
 
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
 
    layer += biases
   
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')

    layer = tf.nn.relu(layer)
 
    return layer

In [28]:
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
 
    return layer

In [29]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

In [30]:
num_channels = 3

filter_size_conv1 = 3
num_filters_conv1 = 8

filter_size_conv2 = 3
num_filters_conv2 = 16

filter_size_conv3 = 3
num_filters_conv3 = 64

fc_layer_size = 128

In [31]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=10,
                     use_relu=False)

In [32]:
y_pred = tf.nn.softmax(layer_fc2,name="y_pred")

In [33]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [34]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)*100

In [35]:
is_correct = tf.equal(y_pred_cls, tf.argmax(y_true,1))

accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [36]:
init = tf.global_variables_initializer()

In [78]:
lr = tf.placeholder(tf.float32)

lrmin = 0.0001
lrmax = 0.003

optimizer = tf.train.GradientDescentOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

sess = tf.Session()
sess.run(init)

batch_size = 10

error = []

for i in range(5000):
    
    ll = i*batch_size
    ul = ll + batch_size

    batch_X, batch_Y = x_train_rs[ll:ul], y_train[ll:ul]
#     batch_X, batch_Y = x_train_rs[i:i+1], y_train[i:i+1]
    
    
    learning_rate = lrmin + (lrmax - lrmin) * math.exp(-i / 5000)
#     learning_rate = 0.001

    train_data = {x: batch_X, y_true: batch_Y, lr: learning_rate}

    sess.run(train_step, feed_dict = train_data)

    train_acc, train_entropy, train_err, pred = sess.run([accuracy, cross_entropy, cost, y_pred], feed_dict=train_data)
    error.append(train_err)
    
    if not i%100:
        print('iteration {} acc : {} error : {}'.format(i+1,train_acc, train_err))
#         print(pred)


test_data = {x: x_test_rs, y_true: y_test}
test_acc, test_entropy = sess.run([accuracy,cross_entropy],feed_dict=test_data)

print('Model has test accuracy of {}'.format(test_acc))

iteration 1 acc : 0.20000000298 error : 2032.21813965
iteration 101 acc : 0.40000000596 error : 164.139678955
iteration 201 acc : 0.300000011921 error : 215.76399231
iteration 301 acc : 0.40000000596 error : 176.852157593
iteration 401 acc : 0.300000011921 error : 185.651077271
iteration 501 acc : 0.20000000298 error : 179.162414551
iteration 601 acc : 0.800000011921 error : 138.081176758
iteration 701 acc : 0.40000000596 error : 174.09387207
iteration 801 acc : 0.699999988079 error : 144.926818848
iteration 901 acc : 0.20000000298 error : 203.066680908
iteration 1001 acc : 0.800000011921 error : 111.487510681
iteration 1101 acc : 0.699999988079 error : 125.906944275
iteration 1201 acc : 0.699999988079 error : 118.702186584
iteration 1301 acc : 0.699999988079 error : 133.646484375
iteration 1401 acc : 0.699999988079 error : 122.104095459
iteration 1501 acc : 0.40000000596 error : 147.484741211
iteration 1601 acc : 0.600000023842 error : 132.5284729
iteration 1701 acc : 0.600000023842 e

### Binary network

In [137]:
def norm1(x):
    x = tf.abs(x)
    x = tf.reduce_mean(x, axis=0)
    x = tf.reduce_mean(x, axis=0)
    return x
    

def get_a_and_B(weights):
    
    w, h, c, n = weights.shape # 3, 3, rgb, 8
        
    bin_weights = tf.sign(weights)
    
    a = norm1(weights)
    
    B = tf.multiply(a, bin_weights)
    
    return a, B

def get_K_and_H(I):
    
    _, w, h, c = I.shape
    w, h = int(w), int(h)
    
    H = tf.sign(I)
    
    A = tf.reduce_sum(I, axis=3)
    A = tf.reshape(A, [-1, w, h, 1 ])
    k = tf.constant(1./(w*h), shape=[w,h,1,1])
    
    K = tf.nn.conv2d(input=A,
                     filter=k,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
    
    return K, H
    

In [32]:
# (H*B).Ka

def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def conv(I, w, h, c, n): 

    weights = create_weights([w, h, c, n])

    a, B = get_a_and_B(weights)
    
    K, H = get_K_and_H(I)
    
    #BinActiv
    HB = tf.nn.conv2d(input=H,
                      filter=B,
                      strides=[1, 1, 1, 1],
                      padding='SAME')
    
    #BinConv
    HBK = tf.nn.conv2d(input=HB,
                       filter=K,
                       strides=[1, 1, 1, 1],
                       padding='SAME')
    
    #BinPool
    pooled = tf.nn.max_pool(value=HBK,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    return pooled

In [75]:
# 3 binary conv layers, flatten layer, 2 fully connected layer

l1 = conv(I, w=3, h=3,c=3, n=8)
l2 = conv(l1, w=3, h=3, c=8, n=16)
l3 = conv(l2, w=3, h=3, c=64, n=64)

l4 = flatten(l3)

l5 = fc(l4, 128)
l6 = fc(10)

(Dimension(3), Dimension(3), Dimension(3), Dimension(8), TensorShape([Dimension(24), Dimension(3), Dimension(3), Dimension(1)]), TensorShape([Dimension(3), Dimension(1)]))
(24, 3, 3, 1)


ValueError: Dimensions must be equal, but are 1 and 32 for 'Conv2D_14' (op: 'Conv2D') with input shapes: [?,32,32,1], [?,32,32,1].

In [112]:
y_pred = tf.nn.softmax(l6,name="y_pred")

y_pred_cls = tf.argmax(y_pred, axis=1)

In [80]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=l6,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)*100

In [113]:
is_correct = tf.equal(y_pred_cls, tf.argmax(y_true,1))

accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

In [ ]:
init = tf.global_variables_initializer()

In [147]:
w = create_weights([3,3,1,2])
# w = tf.constant(2, shape=[3,3,3,8])

In [148]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
bin_weights = tf.sign(w)
a = norm1(w)
print(sess.run([a, w, a*w]))

[array([[ 0.07360779,  0.08817574]], dtype=float32), array([[[[-0.12319851, -0.12676463]],

        [[ 0.0536567 , -0.0322381 ]],

        [[ 0.10264211,  0.00887774]]],


       [[[-0.06670819, -0.07864468]],

        [[ 0.10590909, -0.0497191 ]],

        [[ 0.01875779, -0.03142869]]],


       [[[-0.12760289, -0.19341318]],

        [[-0.05665344, -0.09999122]],

        [[ 0.00734139,  0.1725044 ]]]], dtype=float32), array([[[[-0.00906837, -0.01117756]],

        [[ 0.00394955, -0.00284262]],

        [[ 0.00755526,  0.0007828 ]]],


       [[[-0.00491024, -0.00693455]],

        [[ 0.00779573, -0.00438402]],

        [[ 0.00138072, -0.00277125]]],


       [[[-0.00939257, -0.01705435]],

        [[-0.00417013, -0.0088168 ]],

        [[ 0.00054038,  0.0152107 ]]]], dtype=float32)]


In [138]:
def norm1(x):
    x = tf.abs(x)
    x = tf.reduce_mean(x, axis=0)
    x = tf.reduce_mean(x, axis=0)
    return x